In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pprint

In [ ]:
def open_table_list_columns(table):
    table = pd.read_csv('/volumes/ellieking/Checked/'+table+'.csv', dtype=object)
    pp = pprint.PrettyPrinter()
    pp.pprint(list(table.columns))
    return table

In [ ]:
discharge_codes_df = pd.read_csv('/Users/ellieking/Downloads/discharge_codes - Sheet1.csv', dtype=object)

In [ ]:
discharge_codes = pd.Series(discharge_codes_df.Text.values,index=discharge_codes_df.Code).to_dict()

In [ ]:
discharge_codes

In [ ]:
client = open_table_list_columns('Client')

In [ ]:
client.head()

In [ ]:
client.shape

In [ ]:
client.Serial.nunique()

In [ ]:
client.groupby('Status').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 5))

In [ ]:
client.groupby('Service').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 20))

In [ ]:
client.groupby('ProjectCode').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 20))

In [ ]:
client.groupby('Gender').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 5))

In [ ]:
client.groupby('RelationshipStatus').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 5))

In [ ]:
client.groupby('Ethnic_Origin').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 5))

In [ ]:
client.groupby('Religion').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 5))

In [ ]:
client.groupby('Sexuality').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 5))

In [ ]:
client.groupby('NoFixedAbode').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 5))

In [ ]:
client.groupby('Alert').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 5))

In [ ]:
client.groupby('Scrip_Index').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 5))

In [ ]:
client.CaseNumber.nunique()

In [ ]:
discharge = open_table_list_columns('Discharge')

In [ ]:
discharge.head()

In [ ]:
discharge['discharge_reason'] = discharge['ReasonForClosure'].map(discharge_codes)

In [ ]:
discharge.shape

In [ ]:
discharge.Serial.nunique()

In [ ]:
discharge.groupby('discharge_reason').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 40))

In [ ]:
discharge.Serial.nunique()

In [ ]:
discharge.shape

only one reason per episode?

In [ ]:
sum(discharge.groupby(['Serial', 'Episode']).size() > 1)

think so...

In [ ]:
discharge_gr = discharge.groupby('discharge_reason').size().sort_values(ascending=True)
discharge_reason['frequency'] = pd.DataFrame(discharge_gr) 
discharge_reason['percent'] = discharge_gr / discharge.shape[0]*100


In [ ]:
discharge_reason[['frequency', 'percent']]

In [ ]:
discharge.groupby('DischargeDestination').size().sort_values(ascending=True).plot(kind = 'barh', figsize=(10, 5))

In [ ]:
discharge.groupby('NDTMSReasonForClosure').size().sort_values(ascending=True)

In [ ]:
discharge.NDTMSReasonForClosure.nunique()

In [ ]:
discharge.ReasonForClosure.nunique()

In [ ]:
episode = open_table_list_columns('Episode')

In [ ]:
episode.head()

Tier2 = structured treatmnet -regular 1to1 sessions
Tier3 = unstructured -dropin to group work
Tier4 = signposting

In [ ]:
episode.shape

In [ ]:
episode.Serial.nunique()

In [ ]:
sum(episode.EndDate.isna())

In [ ]:
sum(episode.EndDate.isna())/episode.Serial.nunique()

Less than 10% of episodes with missing end date (presumed to be in ongoing treatment?)

In [ ]:
type(episode.EndDate[0])

In [ ]:
episode['start_date'] = pd.to_datetime(episode.StartDate)

In [ ]:
episode['end_date'] = pd.to_datetime(episode.EndDate)

In [ ]:
episode['episode_duration'] = episode.end_date - episode.start_date

In [ ]:
episode.head()

In [ ]:
episode['Tier2'].value_counts()

In [ ]:
episode['Tier2'].value_counts(normalize=True)

In [ ]:
episode['Tier3'].value_counts()

In [ ]:
episode['Tier3'].value_counts(normalize=True)

In [ ]:
episode['Tier4'].value_counts()

In [ ]:
episode['Tier4'].value_counts(normalize=True)

In [ ]:
35+41

In [ ]:
episode.Tier2.dtype

What does it mean if all Tiers are 0?

In [ ]:
len(episode[(episode['Tier2'] == '0') & (episode['Tier3'] == '0') & (episode['Tier4'] == '0')])

In [ ]:
len(episode[(episode['Tier2'] == '0') & (episode['Tier3'] == '0') & (episode['Tier4'] == '0')])/episode.shape[0] *100

In [ ]:
38 + 41 + 35 

In [ ]:
len(episode[(episode['Tier2'] == '1') & (episode['Tier3'] == '1')])/episode.shape[0] *100

In [ ]:
type(episode['episode_duration'][0])

In [ ]:
len(episode[episode['episode_duration'] < pd.Timedelta(0)])

In [ ]:
print("percent of episodes with start date after end date: {}".format(len(episode[episode['episode_duration'] < pd.Timedelta(0)])/episode.shape[0] *100))

537 episodes have a start date after the end date. These could be:  
1) removed from data  
2) reversed, assuming data entry
3) Manually checked and errors determined
4) checked with source for data entry errors


For now remove these episodes as they only represent 0.19% of data

In [ ]:
episode = episode[episode['episode_duration'] > pd.Timedelta(0)]

In [ ]:
episode['episode_duration'].describe()

In [ ]:
(episode['episode_duration'] / pd.Timedelta(days=1)).hist(bins =100)

In [ ]:
len(episode[episode['episode_duration'] > pd.Timedelta(weeks=520)])

In [ ]:
(episode['episode_duration'] / pd.Timedelta(days=1)).hist(bins =100)

In [ ]:
last3years = episode[episode['start_date'] > '2015-01-01']

In [ ]:
last3years.shape

In [ ]:
len(last3years[(last3years['Tier2'] == '1') & (last3years['Tier3'] == '1')])/last3years.shape[0] *100

In [ ]:
careplan = open_table_list_columns('Careplan')

In [ ]:
careplan.head()

In [ ]:
events1 = open_table_list_columns('Events_01')

In [ ]:
events1

In [ ]:
events2 =  open_table_list_columns('Events_02')

In [ ]:
events2.shape

In [ ]:
events_join = pd.merge(events1,
                         events2,
                         on=['Serial', 'EventID'], 
                         how='outer', 
                         indicator=True, 
                         validate="1:1"
                        )

In [ ]:
events3 = open_table_list_columns('Events_03')

In [ ]:
events3.shape

In [ ]:
harm_reduction = open_table_list_columns('HarmReduction')

In [ ]:
harm_reduction.head()

In [ ]:
modality = open_table_list_columns('Modality')

In [ ]:
modality.head()

In [ ]:
referral = open_table_list_columns('Referral')

In [ ]:
referral.shape

In [ ]:
referral.head()

In [ ]:
risk = open_table_list_columns('RiskAssessment')

In [ ]:
risk.head()

In [ ]:
scripts = open_table_list_columns('Scripts_3')

In [ ]:
tag_manager = open_table_list_columns('TagManager')

In [ ]:
tag_manager.head()

In [ ]:
top = open_table_list_columns('TOP')

In [ ]:
top.head()

In [ ]:
top_plus = open_table_list_columns('TOPPlus')

In [ ]:
top_plus.head()

In [ ]:
open_table_list_columns('ClientInformationReview')

In [ ]:
open_table_list_columns('ClientInjectingHistory')

In [ ]:
open_table_list_columns('ClientSocial')

In [ ]:
open_table_list_columns('ClientSubstanceAnalysis')

In [ ]:
open_table_list_columns('ClientSubstanceType')

In [ ]:
events_types1 = open_table_list_columns('EventsTypes_01')

In [ ]:
events_types2 = open_table_list_columns('EventsTypes_02')